In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df= pd.read_csv('/content/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
X= df.drop(columns=['RowNumber' ,'CustomerId', 'Surname', 'Exited' ], axis=1)

In [6]:
Y= df['Exited']

In [7]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [8]:
Y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [9]:
X['Gender']=X['Gender'].replace({'Female':1,'Male':0})

In [10]:
geography= pd.get_dummies(X['Geography'])

In [11]:
X= pd.concat([X,geography], axis=1)

In [12]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,France,1,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,France,1,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,France,1,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0,0,1


In [14]:
X=X.drop('Geography',axis=1)

In [15]:
 from sklearn.model_selection import train_test_split
 Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.2, random_state= 0)

In [24]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [25]:
 Xtrain= sc.fit_transform(Xtrain)
 Xtest= sc.fit_transform(Xtest)

In [26]:
import tensorflow as tf
from tensorflow import keras

In [27]:
Xtrain.shape

(8000, 12)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dense(6, activation='relu'),
     keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(Xtrain, Ytrain, epochs=50)

In [31]:
model.evaluate(Xtest,Ytest)

63/63 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8575


[0.3358537256717682, 0.8575000166893005]

In [32]:
ypred= model.predict(Xtest)
ypred= (ypred>0.5)

In [33]:
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Ytest,ypred)

In [34]:
cm

array([[1513,   82],
       [ 203,  202]])

In [35]:
from sklearn.metrics import accuracy_score
score= accuracy_score(ypred, Ytest)
score

0.8575

In [36]:
from sklearn.metrics import precision_score
score= precision_score(ypred, Ytest)
score

0.49876543209876545

In [37]:
from sklearn.metrics import recall_score
score= recall_score(ypred, Ytest)
score

0.7112676056338029

In [38]:
from sklearn.metrics import f1_score
score= f1_score(ypred, Ytest)
score

0.5863570391872278

In [39]:
from sklearn.metrics import classification_report

In [48]:
classes= ['Exited','Not Exited']
report= classification_report(Ytest, ypred,target_names=classes)

In [50]:
print(report)


              precision    recall  f1-score   support

      Exited       0.88      0.95      0.91      1595
  Not Exited       0.71      0.50      0.59       405

    accuracy                           0.86      2000
   macro avg       0.80      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000

